# Занятие 6

## Задание 1

Расставьте соответствие между названиями задач и их формальной постановкой:
 
**Правильный ответ:**

Детекция объектов это обвести объекты на изображении ограничивающими прямоугольниками и классифицировать.

Семантическая сегментация это классифицировать каждый пиксель изображения.

Сегментация объектов это классифицировать каждый пиксель изображения и разделить объекты одного класса.

## Задание 2

Посчитайте меру Жаккара для класса 0, для класса 1 и для класса 2. Посчитайте среднее, округлите до 4 знаков после запятой.

Две картинки...

**Правильный ответ:**

Мера Жаккара:

$$J_k(y, a) = \dfrac{\sum\limits_{i=1}^{n}[y_i = k][a_i = k]}{\sum\limits_{i=1}^{n}\max([y_i = k], [a_i = k])}$$

То есть нам нужно для классов 0, 1 и 2 посчитать количество пикселей, у которых совпадает ответ модели с правильным ответом, а также посчитать сколько всего пикселей содержит этот класс, хотя бы от прогноза модели или правильного ответа. Ну или посчитать пересечение масок вида `[img == C]` и объединение масок `[img == C]`.

Для класса 0: пересечение 0, объединение 12
Для класса 1: пересечение 5, объединение 13
Для класса 2: пересечение 4, объединение 16

Итого: (5/13 + 4/16) / 3 = 0.2115 с округлением до 4 знаков.

## Задание 3

Чем идейно плоха архитектура Fully Convolutional Network для задачи сегментации? Выберите все подходящие варианты:

 1. Слишком много параметров, легко переобучается
 2. Используется макс-пулинг, теряем информацию про то, где объект расположен
 3. Тензор с последнего слоя слишком маленький
 4. Используется устаревшая архитектура AlexNet

**Правильный ответ:** все кроме первого верно.

## Задание 4

Усложните модель `UNET` с семинара, обучите ее на датасете OXFORD-PETS, добейтесь попиксельной Accuracy в 88%. Для этого вам понадобится добавить в нее еще блоков вниз и блоков вверх, а также возможно увеличить `base_channels`.

Используйте следующий трансформ для изображений:

```
transform = T.Compose(
    [
        T.Resize((256, 256)),
        T.ToTensor(),
    ]
)
```

Сдайте свои предсказания для тестовой выборки этого датасета (`split='test'`). Сделайте предсказания для следующих объектов:

```
np.random.seed(100)
idx = np.random.randint(len(valid_dataset), size=200)
```

Загрузите свои предсказания в чекер, воспользуйтесь функциями `torch.save`, ваш тензор с предсказаниями должен иметь размер `[200, 1, 256, 256]`.

**Правильный ответ:**

Сделаем все как в задании:

In [ ]:
import torch.nn as nn


def conv_plus_conv(in_channels: int, out_channels: int):
    """
    Makes UNet block
    :param in_channels: input channels
    :param out_channels: output channels
    :return: UNet block
    """
    return nn.Sequential(
        nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.BatchNorm2d(num_features=out_channels),
        nn.LeakyReLU(0.2),
        nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.BatchNorm2d(num_features=out_channels),
        nn.LeakyReLU(0.2),
    )


class UNET(nn.Module):
    def __init__(self):
        super().__init__()

        base_channels = 32

        self.down1 = conv_plus_conv(3, base_channels)
        self.down2 = conv_plus_conv(base_channels, base_channels * 2)
        self.down3 = conv_plus_conv(base_channels * 2, base_channels * 4)
        self.down4 = conv_plus_conv(base_channels * 4, base_channels * 8)
        self.down5 = conv_plus_conv(base_channels * 8, base_channels * 16)

        self.up1 = conv_plus_conv(base_channels * 2, base_channels)
        self.up2 = conv_plus_conv(base_channels * 4, base_channels)
        self.up3 = conv_plus_conv(base_channels * 8, base_channels * 2)
        self.up4 = conv_plus_conv(base_channels * 16, base_channels * 4)
        self.up5 = conv_plus_conv(base_channels * 32, base_channels * 8)

        self.bottleneck = conv_plus_conv(base_channels * 16, base_channels * 16)

        self.out = nn.Conv2d(in_channels=base_channels, out_channels=3, kernel_size=1)

        self.downsample = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        # x.shape = (N, N, 3)

        residual1 = self.down1(x)  # x.shape: (N, N, 3) -> (N, N, base_channels)
        x = self.downsample(residual1)  # x.shape: (N, N, base_channels) -> (N // 2, N // 2, base_channels)

        residual2 = self.down2(x)  # x.shape: (N // 2, N // 2, base_channels) -> (N // 2, N // 2, base_channels * 2)
        x = self.downsample(residual2)  # x.shape: (N // 2, N // 2, base_channels * 2) -> (N // 4, N // 4, base_channels * 2)

        residual3 = self.down3(x)
        x = self.downsample(residual3)

        residual4 = self.down4(x)
        x = self.downsample(residual4)

        residual5 = self.down5(x)
        x = self.downsample(residual5)

        # LATENT SPACE DIMENSION DIM = N // 4
        # SOME MANIPULATION MAYBE
        x = self.bottleneck(x)  # x.shape: (N // 4, N // 4, base_channels * 2) -> (N // 4, N // 4, base_channels * 2)
        # SOME MANIPULATION MAYBE
        # LATENT SPACE DIMENSION DIM = N // 4

        x = nn.functional.interpolate(x, scale_factor=2)
        x = torch.cat((x, residual5), dim=1)
        x = self.up5(x)

        x = nn.functional.interpolate(x, scale_factor=2)
        x = torch.cat((x, residual4), dim=1)
        x = self.up4(x)

        x = nn.functional.interpolate(x, scale_factor=2)
        x = torch.cat((x, residual3), dim=1)
        x = self.up3(x)

        x = nn.functional.interpolate(x, scale_factor=2)  # x.shape: (N // 4, N // 4, base_channels * 2) -> (N // 2, N // 2, base_channels * 2)
        x = torch.cat((x, residual2), dim=1)  # x.shape: (N // 2, N // 2, base_channels * 2) -> (N // 2, N // 2, base_channels * 4)
        x = self.up2(x)  # x.shape: (N // 2, N // 2, base_channels * 4) -> (N // 2, N // 2, base_channels)

        x = nn.functional.interpolate(x, scale_factor=2)  # x.shape: (N // 2, N // 2, base_channels) -> (N, N, base_channels)
        x = torch.cat((x, residual1), dim=1)  # x.shape: (N, N, base_channels) -> (N, N, base_channels * 2)
        x = self.up1(x)  # x.shape: (N, N, base_channels * 2) -> (N, N, base_channels)

        x = self.out(x)  # x.shape: (N, N, base_channels) -> (N, N, 3)

        return x

In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
from IPython.display import clear_output
from torch.optim import Adam
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torchvision.datasets import OxfordIIITPet
from tqdm import tqdm

def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def train(
    model: nn.Module,
    data_loader: DataLoader,
    optimizer: Optimizer,
    loss_fn,
    device: torch.device,
):
    model.train()

    train_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(data_loader, desc='Train'):
        bs = y.size(0)

        x, y = x.to(device), y.squeeze(1).to(device)

        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output.reshape(bs, 3, -1), y.reshape(bs, -1))

        train_loss += loss.item()

        loss.backward()

        optimizer.step()

        _, y_pred = output.max(dim=1)
        total += y.size(0) * y.size(1) * y.size(2)
        correct += (y == y_pred).sum().item()

    train_loss /= len(data_loader)
    accuracy = correct / total

    return train_loss, accuracy


@torch.inference_mode()
def evaluate(
    model: nn.Module, data_loader: DataLoader, loss_fn, device: torch.device
):
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(data_loader, desc='Evaluation'):
        bs = y.size(0)

        x, y = x.to(device), y.squeeze(1).to(device)

        output = model(x)

        loss = loss_fn(output.reshape(bs, 3, -1), y.reshape(bs, -1))

        total_loss += loss.item()

        _, y_pred = output.max(dim=1)
        total += y.size(0) * y.size(1) * y.size(2)
        correct += (y == y_pred).sum().item()

    total_loss /= len(data_loader)
    accuracy = correct / total

    return total_loss, accuracy


def plot_stats(
    train_loss: list[float],
    valid_loss: list[float],
    train_accuracy: list[float],
    valid_accuracy: list[float],
    title: str
):
    plt.figure(figsize=(16, 8))

    plt.title(title + ' loss')

    plt.plot(train_loss, label='Train loss')
    plt.plot(valid_loss, label='Valid loss')
    plt.legend()
    plt.grid()

    plt.show()

    plt.figure(figsize=(16, 8))

    plt.title(title + ' accuracy')
    
    plt.plot(train_accuracy, label='Train accuracy')
    plt.plot(valid_accuracy, label='Valid accuracy')
    plt.legend()
    plt.grid()

    plt.show()
    

def whole_train_valid_cycle(
    model, train_loader, valid_loader, optimizer, loss_fn, device, threshold, title
):
    train_loss_history, valid_loss_history = [], []
    train_accuracy_history, valid_accuracy_history = [], []

    for epoch in range(100):
        train_loss, train_accuracy = train(
            model, train_loader, optimizer, loss_fn, device
        )
        valid_loss, valid_accuracy = evaluate(model, valid_loader, loss_fn, device)

        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)

        train_accuracy_history.append(train_accuracy)
        valid_accuracy_history.append(valid_accuracy)

        clear_output(wait=True)

        plot_stats(
            train_loss_history,
            valid_loss_history,
            train_accuracy_history,
            valid_accuracy_history,
            title,
        )

        if valid_accuracy >= threshold:
            break


@torch.inference_mode()
def predict_segmentation(model: nn.Module, loader: DataLoader, device: torch.device):
    model.eval()

    prediction = []

    for x, _ in loader:
        output = model(x.to(device)).cpu()

        prediction.append(torch.argmax(output, dim=1))

    prediction = torch.cat(prediction)

    return prediction


def main(model_class, threshold, title):
    set_seed(0xDEADF00D)

    transform = T.Compose(
        [
            T.Resize((256, 256)),
            T.ToTensor(),
        ]
    )

    target_transform = T.Compose(
        [
            T.Resize((256, 256)),
            T.PILToTensor(),
            T.Lambda(lambda x: (x - 1).long())
        ]
    )

    train_dataset = OxfordIIITPet('/home/jupyter/mnt/datasets/pets', transform=transform, download=True, target_transform=target_transform, target_types='segmentation')
    valid_dataset = OxfordIIITPet('/home/jupyter/mnt/datasets/pets', transform=transform, download=True, split='test', target_transform=target_transform, target_types='segmentation')
    
    np.random.seed(100)
    idx = np.random.randint(len(valid_dataset), size=200).tolist()
    
    valid_dataset = Subset(valid_dataset, idx)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = model_class().to(device)

    optimizer = Adam(model.parameters(), lr=1e-3)

    loss_fn = nn.CrossEntropyLoss()

    whole_train_valid_cycle(
        model, train_loader, valid_loader, optimizer, loss_fn, device, threshold, title
    )

    torch.save(predict_segmentation(model, valid_loader, device).reshape([200, 1, 256, 256]).to(torch.uint8), 'prediction.pt')

In [ ]:
main(UNET, 0.88, 'UNET segmentation')

## Задание 5

Расставьте соответствие:

 1. One-shot detection -> Подходы к решению задачи детекции объектов, когда прямоугольники выделяются и классифицируются одной моделью
 2. MAP -> Метрика для измерения качества детекции
 3. Two-shot detection -> Подходы к решению задачи детекции объектов, когда прямоугольники сначала как-то выделяются, а потом классифицируются
 4. Non-maximum supression -> Метод для прореживания прямоугольников, выданных моделью

## Задание 6

Расположите этапы работы архитектуры R-CNN в порядке их применения:

1. Выделить области-кандидаты из изображения
2. Посчитать признаки с помощью сверточной сети
3. Классификация с помощью SVM